In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
import numpy as np
import pandas as pd
#from sklearn import preprocessing, model_selection
from sklearn import model_selection
#from scipy.signal import butter,filtfilt,iirnotch,hilbert
from scipy.signal import butter,filtfilt
#from PyEMD import EMD
#import wandb
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
import scipy.io as sio
from tqdm import tqdm
import pandas as pd


In [14]:
import h5py
import numpy as np
import os
import scipy.io  # For loading .mat files
from glob import glob
import matplotlib.pyplot as plt

# Sampling rate "Hz"
fs = 2000

# Desired sEMG Segments Length in seconds
signal_segment_starting = 1 # amount of time delay after cue
signal_segment_ending = 6 # amount of time after signal. 6 seconds is the end of the cue, so 5 or 6 can be used

# Get sEMG Records directory
current_folder = './OzdemirEMG/'  # Change with current folder
Base = os.path.join(current_folder, 'sEMG-dataset/raw/mat')  # Change raw or filtered as needed
Files = glob(os.path.join(Base, '**/*.mat'), recursive=True)
# Files.sort()  # Sort files if needed

gesture_names = ['Rest', 'Extension', 'Flexion', 'Ulnar_Deviation', 'Radial_Deviation', 'Grip', 'Abduction', 'Adduction', 'Supination', 'Pronation']

foldername = 'DatasetsProcessed_hdf5/OzdemirEMG/'
# Automated segmentation and saving of all participant sEMG data to sEMG gesture segment in HDF5
for file_path in Files:
    mat_data = scipy.io.loadmat(file_path)
    data = mat_data['data']
    participant_id = mat_data['iD'][0][0]  # Modify based on where ID is stored in your .mat

    # Create or open HDF5 file for each participant
    hdf5_filename = f'participant_{participant_id}.hdf5'
    hdf5_filename = os.path.join(foldername, hdf5_filename)
    # Make folders if they don't exist
    os.makedirs(os.path.dirname(hdf5_filename), exist_ok=True)
    # Open the file
    with h5py.File(hdf5_filename, 'a') as hdf_file:  # 'a' mode to append if already exists
        
        for rep in range(5):  # 5 repetitions
            rep_coeff = [4, 138, 272, 406, 540][rep]

            for gesture in range(10):  # Total of 10 hand gestures
                start_idx = (signal_segment_starting + rep_coeff + (gesture * 10)) * fs
                end_idx = ((rep_coeff + (gesture * 10)) + signal_segment_ending) * fs
                
                # Multi-channel sEMG data
                multi_channel_sEMG_data = data[start_idx:end_idx, :].T

                # Define a group for each gesture and cycle
                group_name = f'Cycle{rep + 1}/Gesture{gesture_names[gesture]}'
                grp = hdf_file.require_group(group_name)

                # Save the data in the group, formatted as (CYCLE, CHANNEL, TIME)
                grp.create_dataset('sEMG', data=multi_channel_sEMG_data, compression="gzip")

    print(f"Data for participant {participant_id} and all gestures saved in {hdf5_filename}.")

Data for participant 9 and all gestures saved in DatasetsProcessed_hdf5/OzdemirEMG/participant_9.hdf5.
Data for participant 6 and all gestures saved in DatasetsProcessed_hdf5/OzdemirEMG/participant_6.hdf5.
Data for participant 37 and all gestures saved in DatasetsProcessed_hdf5/OzdemirEMG/participant_37.hdf5.
Data for participant 32 and all gestures saved in DatasetsProcessed_hdf5/OzdemirEMG/participant_32.hdf5.
Data for participant 14 and all gestures saved in DatasetsProcessed_hdf5/OzdemirEMG/participant_14.hdf5.
Data for participant 13 and all gestures saved in DatasetsProcessed_hdf5/OzdemirEMG/participant_13.hdf5.
Data for participant 22 and all gestures saved in DatasetsProcessed_hdf5/OzdemirEMG/participant_22.hdf5.
Data for participant 7 and all gestures saved in DatasetsProcessed_hdf5/OzdemirEMG/participant_7.hdf5.
Data for participant 15 and all gestures saved in DatasetsProcessed_hdf5/OzdemirEMG/participant_15.hdf5.
Data for participant 38 and all gestures saved in DatasetsPro

In [34]:
#Flattened dataset with gestures as groups and numpy arrays of shape (CYCLE, CHANNEL, TIME)

for i in range(1, 41):

    hdf5_input_filename = f'participant_{i}.hdf5'
    hdf5_input_filename = os.path.join(foldername, hdf5_input_filename)
    hdf5_output_filename = f'flattened_participant_{i}.hdf5'
    hdf5_output_filename = os.path.join(foldername, hdf5_output_filename)

    # Open the existing HDF5 file to read
    with h5py.File(hdf5_input_filename, 'r') as hdf_read:
        
        # Open a new HDF5 file to write reprocessed data
        with h5py.File(hdf5_output_filename, 'w') as hdf_write:
            
            # Iterate through each gesture (assuming 10 gestures)
            for gesture in range(10):
                all_cycles_data = []  # List to hold data from all cycles for this gesture

                # Collect data from each cycle for this gesture
                for rep in range(1, 6):  # Assuming 5 cycles, named as 'Cycle1', 'Cycle2', ...
                    cycle_group_name = f'Cycle{rep}/Gesture{gesture_names[gesture]}'
                    if cycle_group_name in hdf_read:
                        # Read and append the data
                        cycle_data = hdf_read[cycle_group_name]['sEMG'][:]
                        all_cycles_data.append(cycle_data)

                # Aggregate all cycles data into a single array (CYCLE, CHANNELS, TIME)
                if all_cycles_data:  # Ensure there is data to process
                    aggregated_data = np.stack(all_cycles_data, axis=0)  # Stack along new axis for cycles

                    # Write the aggregated data to the new file
                    hdf_write.create_dataset(f'Gesture{gesture_names[gesture]}', data=aggregated_data, compression="gzip")

    print(f"Reprocessed data saved in {hdf5_output_filename}.")

Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_1.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_2.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_3.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_4.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_5.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_6.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_7.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_8.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_9.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_10.hdf5.
Reprocessed data saved in DatasetsProcessed_hdf5/OzdemirEMG/flattened_participant_11.hdf5.
Reproces

In [36]:
# Check flattened data
hdf5_filename = 'flattened_participant_3.hdf5'
hdf5_filename = os.path.join(foldername, hdf5_filename)
with h5py.File(hdf5_filename, 'r') as hdf_file:
    # List all groups
    print("Keys: %s" % hdf_file.keys())
    a_group_key = list(hdf_file.keys())[0]

    print(hdf_file['GestureRest'].shape)

Keys: <KeysViewHDF5 ['GestureAbduction', 'GestureAdduction', 'GestureExtension', 'GestureFlexion', 'GestureGrip', 'GesturePronation', 'GestureRadial_Deviation', 'GestureRest', 'GestureSupination', 'GestureUlnar_Deviation']>
(5, 4, 10000)
